# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#|export
import numpy as np
from arch import arch_model
from functools import cached_property

In [ ]:
#|export
class GARCHETTE:
    
    def __init__(self) -> None:
        self.params = None
        self._is_fit = False
        self._sig2_init = None

    def fit(self,y):
        self._y = y
        res = arch_model(y,mean='zero',dist='t').fit(disp='off')
        self.params = res.params
        self._sig2_init = res.conditional_volatility[0]**2
        self._is_fit = True
        return self
    
    @cached_property
    def vs(self):
        ...

    @cached_property
    def std_resid(self):
        return self._y / np.sqrt(self.vs)

    def simulate(self,horizon:int, n_rep=1000):
        ...

In [ ]:
y = np.random.randn(300)
GARCHETTE().fit(y)

<__main__.GARCHETTE>

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()